In [11]:
import pandas as pd
import numpy as np

In [12]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')
smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_75090/1207623996.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')


In [13]:
def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']

    # Calculate magnitudes along x, y, and z axes
    accel_mag_x_y = np.sqrt(accel_x**2 + accel_y**2)
    accel_mag_z = np.abs(accel_z)
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check if there is significant upward or downward motion along the z-axis
            if accel_z > 0 and accel_mag_x_y > accel_mag_z:
                motion_type = "Upward Horizontal Motion"
            elif accel_z < 0 and accel_mag_x_y > accel_mag_z:
                motion_type = "Downward Horizontal Motion"
            else:
                motion_type = "Other Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)

In [14]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Create and fit the LabelEncoder
label_encoder_motion = LabelEncoder()
pivot_data4['motion_encoded'] = label_encoder_motion.fit_transform(pivot_data4['motion_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_motion, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_motion.pkl')

# Get the encodings and decodings
encodings = label_encoder_motion.transform(label_encoder_motion.classes_)
decodings = label_encoder_motion.classes_

# Display the encodings and decodings
for encoding, decoding in zip(encodings, decodings):
    print(f"Encoding: {encoding}, Decoding: {decoding}")

Encoding: 0, Decoding: Downward Horizontal Motion
Encoding: 1, Decoding: Other Moving
Encoding: 2, Decoding: Stationary
Encoding: 3, Decoding: Turning/Tilting
Encoding: 4, Decoding: Upward Horizontal Motion


In [15]:
pivot_data4.fillna(0, inplace=True)

In [16]:
smartphone = smartphone.drop(columns = ['activity_type'])
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])

# Group activity_type_full by timestamp and transform it to align with smartphone
activity_type_grouped = activity_type_full.groupby(smartphone['timestamp'])['activity_type'].apply(lambda x: ','.join(x)).reset_index()

# Merge activity_type_grouped with smartphone based on timestamp
smartphone = pd.merge(smartphone, activity_type_grouped, on='timestamp', how='left')

# Print or further process the updated DataFrame
print(smartphone)

smartphone.fillna(0, inplace=True)

# Filter the DataFrame to keep rows where "activity_type" is not equal to 0
merged_data_s_f = smartphone[smartphone['activity_type'] != 0]
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])

# Extract hour and minute from the timestamp and convert to minutes
merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute

merged_data_s_f_filtered = merged_data_s_f[['timestamp', 'activity_type']]

                     timestamp  step_detector_values  audio_value_1  \
0      2017-06-29 07:45:16.506                   1.0            0.0   
1      2017-06-29 07:45:17.407                   0.0            0.0   
2      2017-06-29 07:45:18.407                   0.0          227.0   
3      2017-06-29 07:45:19.407                   0.0          590.0   
4      2017-06-29 07:45:20.446                   0.0         1724.0   
...                        ...                   ...            ...   
617948 2017-07-13 20:13:33.671                   0.0         1956.0   
617949 2017-07-13 20:13:34.663                   0.0         2673.0   
617950 2017-07-13 20:13:35.663                   0.0         2340.0   
617951 2017-07-13 20:13:36.668                   0.0         2046.0   
617952 2017-07-13 20:13:37.668                   0.0         2441.0   

        audio_value_2  audio_value_3  audio_value_4 activity_type  
0             0.00000            0.0            0.0   Video games  
1          

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_75090/4048118510.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_75090/4048118510.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute


In [17]:
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
pivot_data4['timestamp'] = pd.to_datetime(pivot_data4['timestamp'])
# Extract hour and minute from the timestamp and convert to minutes
pivot_data4['timestamp'] = pivot_data4['timestamp'].dt.hour * 60 + pivot_data4['timestamp'].dt.minute
# Merge the dataframes on the 'timestamp' column
pivot_data4_merged = pivot_data4.merge(merged_data_s_f_filtered, on='timestamp', how='left')
pivot_data4_merged.fillna(0, inplace=True)
pivot_data4_merged_filtered = pivot_data4_merged[pivot_data4_merged['activity_type'] != 0]

In [18]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the LabelEncoder
label_encoder_activities = LabelEncoder()
pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_activities, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_activities.pkl')

# Get unique classes from the LabelEncoder
unique_classes = label_encoder_activities.classes_

# Print unique classes
print("Unique Classes:")
for class_ in unique_classes:
    print(class_)

# Get the encodings and decodings
encodings = label_encoder_activities.transform(label_encoder_activities.classes_)
decodings = label_encoder_activities.classes_

# Display the encodings and decodings
for encoding, decoding in zip(encodings, decodings):
    print(f"Encoding: {encoding}, Decoding: {decoding}")

Unique Classes:
At home
Cooking
Eat
Football
In bus
In computer
In vehicle
Meeting
Movie
On bus stop
Pause
Phone was out of the pocket (forgot)
Picnic 
Running
Shop
Shopping& wearing
Sleep
Took off glasses
Train
Video games
Walk
Walking&party
Watching TV
Work
Encoding: 0, Decoding: At home
Encoding: 1, Decoding: Cooking
Encoding: 2, Decoding: Eat
Encoding: 3, Decoding: Football
Encoding: 4, Decoding: In bus
Encoding: 5, Decoding: In computer
Encoding: 6, Decoding: In vehicle
Encoding: 7, Decoding: Meeting
Encoding: 8, Decoding: Movie
Encoding: 9, Decoding: On bus stop
Encoding: 10, Decoding: Pause
Encoding: 11, Decoding: Phone was out of the pocket (forgot)
Encoding: 12, Decoding: Picnic 
Encoding: 13, Decoding: Running
Encoding: 14, Decoding: Shop
Encoding: 15, Decoding: Shopping& wearing
Encoding: 16, Decoding: Sleep
Encoding: 17, Decoding: Took off glasses
Encoding: 18, Decoding: Train
Encoding: 19, Decoding: Video games
Encoding: 20, Decoding: Walk
Encoding: 21, Decoding: Walking&p

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_75090/3103381586.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])


In [19]:
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Assuming 'pivot_data4' is your DataFrame
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_75090/179481233.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)


In [20]:
pivot_data4_merged_filtered.fillna(0, inplace=True)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_75090/202448761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered.fillna(0, inplace=True)


In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import regularizers
import numpy as np

# Define the classification function for motion activity
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Apply the function to create the 'true_motion' column
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

# Split the data into features (X) and target variable (y)
X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'true_motion', 'timestamp']]
y = pivot_data4_merged_filtered['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data for compatibility with CNN
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Define the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(pivot_data4_merged_filtered['activity_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=20, validation_split=0.2, class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_75090/1887731920.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)


Epoch 1/20
246/246 [==============================] - 4s 6ms/step - loss: 3.0073 - accuracy: 0.1065 - val_loss: 2.3442 - val_accuracy: 0.1841
Epoch 2/20
246/246 [==============================] - 1s 4ms/step - loss: 2.9543 - accuracy: 0.1402 - val_loss: 2.4476 - val_accuracy: 0.0575
Epoch 3/20
246/246 [==============================] - 1s 3ms/step - loss: 2.6215 - accuracy: 0.1226 - val_loss: 2.1611 - val_accuracy: 0.2309
Epoch 4/20
246/246 [==============================] - 2s 6ms/step - loss: 2.4224 - accuracy: 0.1304 - val_loss: 2.2138 - val_accuracy: 0.2274
Epoch 5/20
246/246 [==============================] - 1s 5ms/step - loss: 2.2936 - accuracy: 0.1383 - val_loss: 2.0204 - val_accuracy: 0.1119
Epoch 6/20
246/246 [==============================] - 1s 4ms/step - loss: 2.1969 - accuracy: 0.1443 - val_loss: 2.0008 - val_accuracy: 0.0946
Epoch 7/20
246/246 [==============================] - 1s 5ms/step - loss: 2.0778 - accuracy: 0.1328 - val_loss: 1.9758 - val_accuracy: 0.1312
Epoch 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Split the data into features (X) and target variable (y)
X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'timestamp']]
y = pivot_data4_merged_filtered['true_motion']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4_merged_filtered['activity_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/true_motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/20
246/246 [==============================] - 5s 5ms/step - loss: 2.8335 - accuracy: 0.1233 - val_loss: 1.9436 - val_accuracy: 0.1338
Epoch 2/20
246/246 [==============================] - 1s 4ms/step - loss: 1.7831 - accuracy: 0.2263 - val_loss: 1.3694 - val_accuracy: 0.1526
Epoch 3/20
246/246 [==============================] - 1s 4ms/step - loss: 1.4615 - accuracy: 0.2398 - val_loss: 1.3089 - val_accuracy: 0.1414
Epoch 4/20
246/246 [==============================] - 2s 6ms/step - loss: 1.3216 - accuracy: 0.2336 - val_loss: 1.2744 - val_accuracy: 0.1328
Epoch 5/20
246/246 [==============================] - 1s 4ms/step - loss: 1.2889 - accuracy: 0.2034 - val_loss: 1.2493 - val_accuracy: 0.1368
Epoch 6/20
246/246 [==============================] - 1s 4ms/step - loss: 1.2126 - accuracy: 0.2120 - val_loss: 1.2649 - val_accuracy: 0.1287
Epoch 7/20
246/246 [==============================] - 1s 4ms/step - loss: 1.2127 - accuracy: 0.1941 - val_loss: 1.2610 - val_accuracy: 0.1267
Epoch 

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

# Load the saved model
model_path = "/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/true_motion.keras"
model = load_model(model_path)

# Preprocess the unlabeled data
X_unlabeled_scaled = scaler.transform(pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'timestamp']])

# Make predictions
y_pred_prob = model.predict(X_unlabeled_scaled)

# Assuming your model outputs probabilities and you want to select the class with the highest probability
y_pred = np.argmax(y_pred_prob, axis=1)

# Add the predicted labels to the DataFrame
pivot_data4['true_motion'] = y_pred

5427/5427 [==============================] - 12s 2ms/step


In [29]:
# Load the saved model
model_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
model = load_model(model_path)

# Preprocess the unlabeled data
X_unlabeled_scaled = scaler.transform(pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'true_motion', 'timestamp']])

# Make predictions
y_pred_prob = model.predict(X_unlabeled_scaled)

# Assuming your model outputs probabilities and you want to select the class with the highest probability
y_pred = np.argmax(y_pred_prob, axis=1)

# Add the predicted labels to the DataFrame
pivot_data4['activity_encoded'] = y_pred

5427/5427 [==============================] - 11s 2ms/step
